# Code explaination day after day
## Code review at 28th jully 2019
### Project
This project consists to create an AI that can take out of the DSLR camera pictures and quickly process them without any human intervention. The goal is to automatically remove the noise and boost contrast for galaxies, nebulosities... removing moon or artificial light reflecting effect on clouds or atmosphere.
### Current approach
To build this system I have to make choices and attempts to determine which element better allow the system to perform this image processing task :<br/>
- First I need to choose which type of layer, which component will constitute my system and linked with that how I will gather them to constitute the global AI<br/>
- Then, if we suppose that the architecture can potentially perform the processing task, we need to trained it properly so that it can understand if its correction are contributing to actually create a better image<br/><br/>

In order to better guide the part of the AI that will actually process the input image I added a discriminator following the approach of GANs systems.
But what is a GAN ? A <b>GAN</b> is a <b>generative adversarial network</b>. We can find a first explanation with its name : it is an AI, a <b>network</b> that can <b>generate new data</b> after training on a dataset. The <b>adversarial</b> part refers to the learning process : in fact, this AI is composed of <b>two competitive AI</b>, the generator and the discriminator. The discriminator wants after his training to distinguish a well-processed image from a badly-processed / not processed image. But as the discriminator is training on human processed and non-processed images, half the time the generator comes and suggests a processed-image <b>trying to fool the discriminator</b> and reproduce a real human processed image.
So, my current AI system is composed of a <b>generator</b> who processed the image and a <b>discriminator</b> who will take the processed image of the generator and ouput the probability that this output image has actually been well processed. It will detect residual noise, blur effects.... The benefits can be doubled : first, a well trained discriminator will <b>better analyse</b> the result of the generator and <b>guide</b> it to a correct image process. Indeed, if we want to directly train the generator to achieve the same correction as a human we will use a mean squared quadratic error that will give a single number as a loss measure without telling if it is a white balance problem, a noise, a blur problem... Moreover with a discriminator we can train it to <b>generalize human image processing</b>. With that we can after discriminator training better generalize corrections to do on input images.

After many searches on the web on the current AI structures, I have decided to use specific layers to help it to achieve its goal. In the global AI it leads to this architecture :
In the generator, I used <b>dense</b> layer that perform the following operation : if $I$ is the input 'image' and $O$ the ouput and $K$,$B$ the kernel and bias matrix that have the dimensions of the input image, we have : $O=I \cdot K+B$. It allows that each pixel of the input is directly linked to each pixel of the ouput on the contrary of convolutional layers. I allowed the layer to learn many different characteristics of the noised image (see add layer) that  at the beginning and then reduce it back to 3 characteristics, 3 color channels to retrieve an image.
I added regularization layers : first <b>dropout</b> to randomly disabled a defined proportion of neurons in the previous layer of the network. After that, I added <b>batch normalization</b> layers to uniformize on each group of images that go through the network their ouput and with that eliminating too strong outputs and avoid overfitting (the fact that the network is precisely adapted to the training dataset and makes wrong predictions / image processing for new examples subjected). Then, I added <b>local response normalization</b> layers to disabled neurons that are in the local neighborhoods of neurons strongly activated (that are very important for the result of the layer). It allows to diversify characteristics found by the layer.
I implemented the SELU activation function that allow to auto-contain intermediate values of the network and with that improve the learning process.
At the end of the network I add the input so that if the generator have to process images with finding the right $f$ function to remove noise, blur... then the biggest part of it will have to find the $f-id$ function so the opposite of noise, blur...

After the generator I split the discriminator into two different part : the noise part was charged with detecting noise and the image part was charged with analysing the image itself.
First in the noise part, I used convolution with SELU activation and batch normalization. At the end, in order to detect noise I added a subtract layer so that conventional layers modelize the noise and then remove it with the subtract operation. But on second thougths, it will be more difficult for them to find common characteristics of the noise a random phenomenon... But at least it can performs a similar role as the add layer of the generator because $-(-)=+$ :)
In the image part I used <b>inception</b> module : I implemented the Google layer inception last version that combine different layers in paralell that will simultaneously perform a different type of analysis on their common input. That allows to improve characteristics detection, potentially reduce the number of parameters to learn and globally accelerates the learning process. Moreover in the last version google team decided to strengthen the link between the input and the ouput by adding to the current ouput the input. With that, if we consider that the group of layers inception have to process the image applying it a function f, with this extra adding layer, inception layers before the addition of the input have to apply the function $f - id$.

Then, I <b>'flatten'</b>, I brake image geometry, leaving pixel position information and consider each pixel in a single column (vector) to then come (with a dense layer and a sigmoid activation function) to a value that will be interpreted as the probability of the image discriminator input to be well processed

That's it for the structure part.
Then I had to choose which method will be the best to train my big AI system

### Training process

I had to choose the right metric, error measure to lead the AI and then train a part of the AI or the whole AI.
First, I decided to guide my system with splitting at the beginning the generator and discriminator to pretrain them in order to engage both of them on the right path (good corrections for generator and a global idea of the characteristics of a good and bad image for the discriminator).

For the discriminator part I used the $logcosh$ error function  because in past AI on the same dataset it works well. Keras' documentation explains <i>`log(cosh(x))` is approximately equal to `(x ** 2) / 2` for small x and to `abs(x) - log(2)`</i><b>Think to cosh graph</b><i> for large x. This means that `logcosh` works mostly like the mean squared error, but will not be so strongly affected by the occasional wildly incorrect prediction.</i> At the opposite of the MSE (mean-squared error) with the form $x^{2}$ which is the same for small $x$ but stays with a $x^{2}$ for big $x$ (which increases the importance of potentially incorrect predictions).
Furthermore I used a backup system to avoid nan (no answer : so infinite or problem) problem by :
- resetting the discriminator trained values if it occurs during the first 10 training steps
- reloading last training step backup and retrying else ; I allowed 10 retryings to prevent an infinite retrying loop
- if 10 retryings have been done I stop the training process
For the learning rate I used an exponential planning

For the generator, I used the classical MSE error function

After that I took the whole system and trained it the following way :
- I make a backup from past discriminator and generator-discriminator state
- I trained 3 different times from the original backup the generator followed by the discriminator to have three attempts that return me 3 error values. For that, I use the `binary_crossentropy` <i>Entropy is a measure of the uncertainty associated with a given distribution q(y).</i> I save each one.
- Then for each of these attempts I trained 3 times the discriminator (with `binary_crossentropy`) that return me 3 other error values. I save each one.
- Then, I compute the mean between generator followed by discriminator and discriminator error and sort the list of each generator-discriminator and discriminator combination
- Finally, I pick the two models with the lowest error. With that I reduce chances of one of the 2 AI system to be most important than the other and block it at one high error value

To choose the learning rate I have used the same exponential planning for generator-discriminator and discriminator : I counted on the training algorithm above to avoid one part of the system to block the other as explained.

### Visualize the results and understand if the AI was well-Training

In order to achieve that goal I needed to see / have access to :
- generator-discriminator errors values
- discriminator errors values
- generator-discriminator discriminator mean errors values
- image input
- image if possible (if the geometry was preserved) intermediate analysis results
- image generator output
- corresponding predicted probabilities to be well-processed
- probability of a human-processed image to be well-processed
- probability of a manually-deteriorated-image to be well-processed

Because of keras, for which tensorboard is not 'out-of-the-box-prepared', I used the <b>print option for the errors</b> to follow the training during it, <b>combined with manual summary callings</b> to add error values. I implemented an algorithm to create several subAI that will retrieve the global AI state but output <b>intermediate image-processing steps including generator output...</b>, steps in generator and in discriminator. Then I can submit them to a <b>summary</b> (file thread that will contains the images, error values...). Moreover <b>I wanted to</b> add each <b>probability values</b> in some way to compare to the image result.

### Current problems to solve
- Tensorboard image part is <b>lagging</b> because there is too much tags to the images and globally too much images
- Find a way to integrate probabilities

## 29th July 2019

### 1. A list that will directly contain layer to compute the output
During model build we add selected-layers to the list
During the backup step of the training process, subAIs returns the output of each of these layers

### 2. Remove canal of layers ouput from tags to reduce tensorboard lags

### 3. Add graphics for probabilities

### 4. Compute intermediate results for human-processed image and manually-deteriorated image

### Current problems to solve
- Related to point 1. : one part of one of the subAI in the backup process ouput on nothing and not on the logical following layer of the subAI
- Part of variables are written in French
- Detecting if keras is missing some summary writtings and if yes solve it if possible
## 31 july 2019

### 1. Global AI with one input and multiple outputs corresponding to the desired layers to vizualize

## 01 august 2019

Still trying to build the backup model. But the fact seems to be, that keras is not able to detect layers of generator (??? no problems detected but it might come from the compilation order)  and discriminator (maxpooling2d not recognized as a layer). I "couldy" suspect that keras only sees the top layers models so generator and discriminator even if the fact to have firstly no issues with generator layers (first in the list of the output layers) is strange... -> Test to figure if it is possible to access to sublayers of a model with several groups of layers. Two models of two simple convolutional  layers then grouped sequentially into a single big model. After that we try to build a model that has the single input of the first group and outputs on the output of each last layer of each group.

Found the same issue graph disconnected only for the backup model. Same problem if we keep only one layer --> not coming from keras obviously :)

Trying to specify the appropriate layer thanks to keras methods (model -> list layer...) Seems to be not a problem of adress but of conception : it is impossible to create a model with both to the global input and the layers of a submodel

The final test code : 
```python
Ltest = []
def group1():
    global Ltest
    inpt1 = Input(shape=(199,199,3),name='input1')
    couche = Convolution2D(filters=3,kernel_size=(2,2),padding='same')(inpt1)
    Ltest.append(couche)
    couche = Convolution2D(filters=3,kernel_size=(2,2),padding='same')(couche)
    Ltest.append(couche)
    return Model(input=inpt1, output=couche,name='Group1')
def group2():
    global Ltest
    inpt2 = Input(shape=(199,199,3),name='input2')
    couche = Convolution2D(filters=3,kernel_size=(2,2),padding='same')(inpt2)
    Ltest.append(couche)
    couche = Convolution2D(filters=3,kernel_size=(2,2),padding='same')(couche)
    Ltest.append(couche)
    return Model(input=inpt2, output=couche,name='Group2')
def total(g1,g2):
    print([type(elem) for elem in [g1,g2]])
    inpt = Input(shape = (199, 199, 3),name='inputTot')
    couche = g1(inpt)
    print(type(couche))
    print(couche)
    couche = g2(couche)
    return Model(input=inpt, output=couche)
model = total(group1(),group2())
print('\n\n')
print([layer.name for layer in model.layers])
print([layer.name for layer in model.layers[1].layers])
print('\n\n')
model_backup = Model(input = model.input,output=[model.layers[1].layers[1],model.layers[1].layers[2]])
```

Two options : 
- modify generator and discriminator to directly output intermediate results. This is probably the easiest implementation but with it we lost the seperation between the model to train and the model to visualize. We may loose computing power to extract subimages at each iteration. To that extent, we can backup at each iteration model layers states. However we will probably overload the Tensorboard. 
- The other solution is to completly rebuild both generator and discriminator in a big model which will allow to access to layers because each layers are at the first level of the current training model. But this technic will oblige to repretrain the two parts and use names to properly reconstruct networks

Second option implemented : today still bugs

## 02 august 2019

No more graph disconnected error

## 10 august 2019

Without close and reopen system with 50 epochs training, the images are properly saved. The result is always black and white and often saturated.
The system of tree training filter allows to quickly make the system converging.

The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/04.png) Generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/02.png)

The deteriorated image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/01.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/03.png)

With the training values : 

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars01.JPG)

Two solutions can be used to improve the system : 
- Reduce generator neuron number to avoid it to fool the discriminator
- Use convolutional layers

## 11 august 2019
The 7th AI uses a generator with less neurons. Result more saturated

## 12 august 2019
6th AI restarted to have image results on tensorboard.
The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/08.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/06.png)

The deteriorated one ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/07.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/05.png)

With scalars

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars02.JPG)

For the model with less neurons (100,10,3) we have

The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/012.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/10.png)

The deteriorated one ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/11.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/09.png)

With scalars

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars03.JPG)

Then with convolution layers -> Modele 8

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/gen0.png) 

The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/16.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/14.png)

The deteriorated one ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/15.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/13.png)

With scalars

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars04.JPG)

## 13th august  2019

The result is interesting even if it is a little bit too saturated
With model 8 we try to keep training for 100 epochs for model with 500-100-3 filters and kernels of 2

The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/20.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/18.png)

The deteriorated one ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/19.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/17.png)

With scalars

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars05.JPG)

Attempt with 100,10,3 filters

The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/24.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/22.png)

The deteriorated one ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/23.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/21.png)

With scalars

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars06.JPG)

Back to the 500,100,3 model we try to add a dense layer with 3 neurons at the end

The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/28.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/26.png)

The deteriorated one ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/27.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/25.png)

With scalars

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars07.JPG)

We need to precise the training. For that, I will test to add generator own MSE  loss to gen-disc one. With the same model of 500-100-3 conv + dense 3 neurons. For that we have to create a custom loss function that will make : $loss = binary_crossentropy(true_probability,predicted_probability)+MSE(desired_image,generated_image)$. But this will be impossible to use : desired_image change at each iteration so we would have to recompile at each iteration with a different layer for th desired image

That is why I will use another optimizer with MSE to add the MSE to the trraining

Reviewing th logic of the top 27 training

We will first train 3 versions of the generator with the MSE to approximate the desired image. Then we will train the generator discriminator group to have an more precisely guide the generator. Finally we will train the discriminator to avoid it to be directly fooled by the generator.

So it will be as that with G for generator training, GD for the group generator-discriminator and D for the discriminator.

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/graph_top27_training.gv.png)

So at 
- <b>Generator</b> training : we need to restore at each new training possibility only the <b>generator</b>
- <b>Generator-discriminator</b> training : we need to restore at each new training possibility the <b>generator-discrriminator</b>
- <b>Discriminator</b> training : we need to restore at each new training possibility the <b>generator-discriminator</b>

## 18th august 2019

Bug fixes, trainable
No improvements : average loss 0.3 with 22 steps

Solutions : 
- Increase genrator training parameter

Which lead to 

The original image ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/32.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/30.png)

The deteriorated one ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/31.png) generated ![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/29.png)

With scalars

![](https://github.com/Rob174/Astronomy/raw/Astronomy/AI/Scalars08.JPG)